In [2]:
from Bio.PDB import PDBIO, Structure, Model, Chain, Atom, Residue
from Bio.PDB.PDBIO import Select

import py3Dmol
import fileinput

file_name_5 = "cube_5_tetrahedra"
file_name_6 = "cube_6_tetrahedra"

dispositions = [file_name_5, file_name_6]

In [3]:
# Define cube vertices (pseudoatoms)
atoms = {
    "PS1": (0.0, 0.0, 0.0),
    "PS2": (10.0, 0.0, 0.0),
    "PS3": (0.0, 0.0, 10.0),
    "PS4": (10.0, 0.0, 10.0),
    "PS5": (0.0, 10.0, 0.0),
    "PS6": (10.0, 10.0, 0.0),
    "PS7": (0.0, 10.0, 10.0),
    "PS8": (10.0, 10.0, 10.0),
}

# Define connectivity (edges of the cube)
cube_connectivity = [
    (1, 2), (1, 3), (1, 5),
    (2, 4), (2, 6),
    (3, 4), (3, 7),
    (4, 8),
    (5, 6), (5, 7),
    (6, 8),
    (7, 8),
]

# Define tetrahedra vertices (pseudoatoms)
cube_5_tetrahedra = [
    (1, 3, 4, 7),
    (1, 5, 7, 6),
    (1, 2, 6, 4),
    (4, 6, 7, 8),
    (1, 4, 6, 7),
]

cube_6_tetrahedra = [
    (1,3,4,5),
    (1,2,4,5),
    (2,5,6,4),
    (5,6,4,8),
    (7,8,4,5),
    (7,3,4,5),
]

cube_6_connectivity = [
    (5,3),(1,4),(5,4),
    (5,2),
    (4,6),
    (5,8),
    (4,7),(8,5),
    (7,4),
]

# Number of tetrahedra
print(f"Number of the 5-spilt tetrahedra: {len(cube_5_tetrahedra)}")
print(f"Number of the 6-spilt tetrahedra: {len(cube_6_tetrahedra)}")

def add_connectivity(name):
    tetrahedra_connectivity = []
    print(f"Adding connectivity")

    for tetrahedron in name:
        for i in range(4):
            for j in range(i+1, 4):
                tetrahedra_connectivity.append((tetrahedron[i], tetrahedron[j]))

    return tetrahedra_connectivity

# Create structure
def get_structure(name):
    structure = Structure.Structure(name)
    model = Model.Model(0)
    chain = Chain.Chain("A")
    structure.add(model)
    model.add(chain)

    # Add atoms
    for i, (name, coords) in enumerate(atoms.items(), start=1):
        res = Residue.Residue((" ", i, " "), "PSE", " ")
        atom = Atom.Atom(name, coords, 1.0, 1.0, " ", name, i, "C")
        res.add(atom)
        chain.add(res)

    return structure


def postprocess(stream, connectivity):
    added = []
    """ Add CONECT records """
    for a1, a2 in connectivity:
        added.append((a1, a2))

    set_added = set(added)
    new = list(set_added)

    new.sort()

    for (a1, a2) in new:
        if(a2, a1) in new:
            set_added.remove((a1, a2))

    stream.write("\n")
    for a1, a2 in new:
        stream.write(f"CONECT{a1:5}{a2:5}\n")


Number of the 5-spilt tetrahedra: 5
Number of the 6-spilt tetrahedra: 6


In [4]:
def sort_connectivity(connectivity):
    sorted_list = []
    for a1, a2 in connectivity:
        if(a1 > a2):
            a1, a2 = a2, a1
        sorted_list.append((a1, a2))

    return sorted(set(list(sorted_list)))

def clean(cube_connectivity, tetrahedra_connectivity_sorted):
    for edge in cube_connectivity:
        if edge in tetrahedra_connectivity_sorted:
            tetrahedra_connectivity_sorted.remove(edge)
    
    return tetrahedra_connectivity_sorted

In [13]:
CUBE_ONLY = False

if CUBE_ONLY:
    dispositions = ["cube"]
else:
    dispositions = [file_name_5, file_name_6]

for file_name in dispositions:
    # Create the 8 atoms for the cube
    io = PDBIO()
    io.set_structure(get_structure(file_name))
    io.save(file_name + ".pdb")

    # Read all lines
    with open(file_name + ".pdb", "r") as file:
        lines = file.readlines()

    # Write back only lines that do NOT start with "TER" or "END"
    with open(file_name + ".pdb", "w") as file:
        for line in lines:
            if not (line.startswith("TER") or line.startswith("END")):
                file.write(line)

    print("Lines starting with 'TER' and 'END' have been removed!")

    #open stream
    with open(file_name + ".pdb", "a") as stream:
        # Connect the cube vertices
        postprocess(stream, cube_connectivity)
        if not CUBE_ONLY:
            tetrahedra_connectivity = add_connectivity(eval(file_name))
            tetrahedra_connectivity_sorted = sort_connectivity(tetrahedra_connectivity)
            tetrahedra_connectivity_clean = clean (cube_connectivity, tetrahedra_connectivity_sorted)
            print("tetrahedra_connectivity:\n", tetrahedra_connectivity_sorted)
            postprocess(stream, tetrahedra_connectivity_sorted)
        stream.write("END\n")

    print("PDB file with CONECT records saved as", file_name + ".pdb")



Lines starting with 'TER' and 'END' have been removed!
Adding connectivity
tetrahedra_connectivity:
 [(1, 4), (1, 6), (1, 7), (4, 6), (4, 7), (6, 7)]
PDB file with CONECT records saved as cube_5_tetrahedra.pdb
Lines starting with 'TER' and 'END' have been removed!
Adding connectivity
tetrahedra_connectivity:
 [(1, 4), (2, 5), (3, 5), (4, 5), (4, 6), (4, 7), (5, 8)]
PDB file with CONECT records saved as cube_6_tetrahedra.pdb


In [6]:
for file_name in dispositions:
    # Read a pdb file
    with open((file_name + ".pdb"), "r") as f:
        pdb_data = f.read()

    # Visualize in Jupyter Notebook
    viewer = py3Dmol.view(width=400, height=400)
    viewer.addModel(pdb_data, "pdb")

    # Style stick, add color
    viewer.setStyle({"stick": {'colorscheme': "cyanCarbon"}})

    # Add X, Y, Z axes using cylinders
    axis_length = 20
    axis_radius = 0.2

    # X-axis (Red)
    viewer.addCylinder({
        'start': {'x': 0, 'y': 0, 'z': 0}, 
        'end': {'x': axis_length, 'y': 0, 'z': 0}, 
        'radius': axis_radius, 
        'color': 'red'
    })

    # Y-axis (Green)
    viewer.addCylinder({
        'start': {'x': 0, 'y': 0, 'z': 0}, 
        'end': {'x': 0, 'y': axis_length, 'z': 0}, 
        'radius': axis_radius, 
        'color': 'green'
    })

    # Z-axis (Blue)
    viewer.addCylinder({
        'start': {'x': 0, 'y': 0, 'z': 0}, 
        'end': {'x': 0, 'y': 0, 'z': axis_length}, 
        'radius': axis_radius, 
        'color': 'blue'
    })

    viewer.zoomTo()
    viewer.show()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [7]:
# Sort the calculated by hand connectivity
# print("cube_6_connectivity non sorted:\n", cube_6_connectivity)
cube_6_connectivity_sorted = sort_connectivity(cube_6_connectivity)
print("cube_6_connectivity sorted:\n", cube_6_connectivity_sorted)

# Now take the vertices of the tetrahedra and get to the same result
tetrahedra_connectivity = add_connectivity(cube_6_tetrahedra)
# print("tetrahedra_connectivity:\n", tetrahedra_connectivity)
tetrahedra_connectivity_sorted = sort_connectivity(tetrahedra_connectivity)
# print("tetrahedra_connectivity sorted:\n", tetrahedra_connectivity_sorted)

# Remove the edges part of the cube connectivity
tetrahedra_connectivity_clean = clean(cube_connectivity, tetrahedra_connectivity_sorted)
print("tetrahedra_connectivity_clean:\n", tetrahedra_connectivity_clean)

# Correct!


cube_6_connectivity sorted:
 [(1, 4), (2, 5), (3, 5), (4, 5), (4, 6), (4, 7), (5, 8)]
Adding connectivity
tetrahedra_connectivity_clean:
 [(1, 4), (2, 5), (3, 5), (4, 5), (4, 6), (4, 7), (5, 8)]
